# 당뇨병 회귀 문제 ( pytorch )

## import Libraries

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

## 데이터 전처리 및 스케일링

In [34]:
# 데이터 불러오기
df = pd.read_csv("https://github.com/MyungKyuYi/AI-class/raw/main/diabetes.csv")
print(df.head())
print("\n=====================================================\n")

df = df.drop(columns=['Outcome']) # 회귀 문제로 사용하기 위해 Outcome 컬럼 삭제

# 결측치 확인
print(df.isnull().sum())
print("\n=====================================================\n")

X = df.drop(['BMI'], axis=1).values # Feature
Y = df['BMI'].values.astype(np.float32) # 예측해야 할 column

# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 분할된 데이터의 shape을 출력
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print("\n=====================================================\n")

# numpy 배열 --> torch tensor으로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32).view(-1, 1) # view(-1, 1) : (행, 1) 형태로 변환. 모델이 기대하는 입력 형태는 2차원 배열이기 때문이다.
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32).view(-1, 1)

# torch tensor 데이터의 shape을 출력
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print("\n=====================================================\n")

# PyTorch DataLoader로 감싸서 미니배치 학습 가능하게 함
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # 배치 사이즈는 32로 설정.
test_loader = DataLoader(test_dataset, batch_size=32)

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64


(614, 7)
(614,)
(154, 7)
(154,)


torch.Size([

## 모델 정의

In [37]:
# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential( # 층들을 나열
            nn.Linear(7, 64), # 입력층 : 특성 개수 7
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1) # 회귀 : 출력 뉴런 수 1
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU가 있으면 GPU 사용, 아니면 CPU 사용한다는 뜻
model = RegressionModel().to(device) # 모델 생성
criterion = nn.MSELoss() # 회귀의 손실 함수 : MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 : Adam

## 모델 학습

In [40]:
# 학습 루프
model.train() # 학습 모드
for epoch in range(50): # 50 epoch동안 훈련
    total_loss = 0 # loss의 초기값은 당연히 0으로 설정
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch) # 배치별로 예측 수행. output : 모델이 예측한 실제 수치값(회귀)
        loss = criterion(output, Y_batch) # 예측값(output)과 실제 값(Y_batch)을 매치하여 loss 계산
        loss.backward() # 역방향 전파로 가중치/편향 업데이트
        optimizer.step()
        total_loss += loss.item() # loss를 누적
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}") # epoch마다 loss를 출력

Epoch 1, Loss: 1056.7410
Epoch 2, Loss: 1003.8851
Epoch 3, Loss: 899.8098
Epoch 4, Loss: 754.0852
Epoch 5, Loss: 553.8759
Epoch 6, Loss: 351.5057
Epoch 7, Loss: 187.3017
Epoch 8, Loss: 113.7791
Epoch 9, Loss: 90.9408
Epoch 10, Loss: 83.7394
Epoch 11, Loss: 77.8138
Epoch 12, Loss: 72.4734
Epoch 13, Loss: 78.3855
Epoch 14, Loss: 67.6151
Epoch 15, Loss: 64.2486
Epoch 16, Loss: 63.8296
Epoch 17, Loss: 62.7704
Epoch 18, Loss: 60.0334
Epoch 19, Loss: 60.7900
Epoch 20, Loss: 59.6321
Epoch 21, Loss: 57.6598
Epoch 22, Loss: 55.5847
Epoch 23, Loss: 54.1251
Epoch 24, Loss: 52.7364
Epoch 25, Loss: 53.3504
Epoch 26, Loss: 56.8195
Epoch 27, Loss: 51.7190
Epoch 28, Loss: 49.8781
Epoch 29, Loss: 47.7679
Epoch 30, Loss: 48.1726
Epoch 31, Loss: 47.2711
Epoch 32, Loss: 47.2341
Epoch 33, Loss: 45.1115
Epoch 34, Loss: 45.5323
Epoch 35, Loss: 45.0777
Epoch 36, Loss: 44.4991
Epoch 37, Loss: 43.0839
Epoch 38, Loss: 42.8995
Epoch 39, Loss: 42.6770
Epoch 40, Loss: 42.6106
Epoch 41, Loss: 44.0598
Epoch 42, Loss:

## 모델 평가

In [43]:
# 평가
model.eval() # 평가 모드
preds, actuals = [], []
with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy() # 예측값을 numpy 형태로 변환 후,
        preds.extend(outputs) # preds 리스트에 예측값을 저장
        actuals.extend(Y_batch.numpy()) # actuals 리스트에 실제 값을 저장

mse = mean_squared_error(actuals, preds) # 실제 값과 예측 값의 mse를 계산하고 출력.
print(f"Test MSE: {mse:.4f}")

Test MSE: 63.9847
